<a href="https://colab.research.google.com/github/farisazizy/Artificial-Intelligence/blob/main/Tupro_2_Fuzzy_with_Sugeno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
def trapesium(x, a, b, c, d):
  if x <= a or x >= d:
    return 0
  if a < x and x < b:
    return (x-a)/(b-a)
  if b <= x and x <= d:
    return 1
  if c < x and x <= d: 
    return -(x-d)/(d-c)

In [182]:
def segitiga(x, a, b, c):
  if x <= a or x >= c:
    return 0
  if x == b:
    return 1
  if a < x and x <= b:
    return (x-a)/(b-a)
  if b < x and x <= c:
    return -(x-c)/(c-b)

In [114]:
pelayanan_buruk = lambda x: trapesium(x, 0, 0, 30, 40)
pelayanan_sedang = lambda x: trapesium(x, 30, 40, 50, 60)
pelayanan_baik = lambda x: trapesium(x, 50, 60, 70, 90)
pelayanan_sangatbaik = lambda x: trapesium(x, 70, 90, 101, 101)

In [115]:
makanan_buruk = lambda x: trapesium(x, 0, 0, 3, 5)
makanan_sedang = lambda x: segitiga(x, 3, 5, 7)
makanan_baik = lambda x: trapesium(x, 5, 7, 11, 11)

In [116]:
kelayakan_tinggi = lambda x: trapesium(x, 0, 0, 50, 60)
kelayakan_rendah = lambda x: trapesium(x, 50, 60, 100, 100)

In [176]:
x = 75

pburuk = pelayanan_buruk(x)
psedang = pelayanan_sedang(x)
pbaik = pelayanan_baik(x)
psangatbaik = pelayanan_sangatbaik(x)

In [199]:
y = 6

mburuk = makanan_buruk(y)
msedang = makanan_sedang(y)
mbaik = makanan_baik(y)

In [200]:
# RULES
rendah = 0
tinggi = 0
sedang = 0

#1
if pburuk != 0 and mburuk != 0:
  temp = min(pburuk, mburuk)
  rendah = max(rendah, temp)
#2
if pburuk != 0 and msedang != 0:
  temp = min(pburuk, msedang)
  rendah = max(rendah, temp)
#3
if pburuk != 0 and mbaik != 0:
  temp = min(pburuk, mbaik)
  sedang = max(sedang, temp)
#4
if psedang != 0 and mburuk != 0:
  temp = min(psedang, mburuk)
  rendah = max(rendah, temp)
#5
if psedang != 0 and msedang != 0:
  temp = min(psedang, msedang)
  sedang = max(sedang, temp)
#6
if psedang != 0 and mbaik != 0:
  temp = min(psedang, mbaik)
  sedang = max(sedang, temp)
#7
if pbaik != 0 and mburuk != 0:
  temp = min(pbaik, mburuk)
  rendah = max(rendah, temp)
#8
if pbaik != 0 and msedang != 0:
  temp = min(pbaik, msedang)
  tinggi = max(tinggi, temp)
#9
if pbaik != 0 and mbaik != 0:
  temp = min(pbaik, mbaik)
  tinggi = max(tinggi, temp)
#10
if psangatbaik != 0 and mburuk != 0:
  temp = min(psangatbaik, mburuk)
  sedang = max(sedang, temp)
#11
if psangatbaik != 0 and msedang != 0:
  temp = min(psangatbaik, mburuk)
  tinggi = max(tinggi, temp)
#12
if psangatbaik != 0 and mbaik != 0:
  temp = min(psangatbaik, mbaik)
  tinggi = max(tinggi, temp)


In [201]:
rendah

0

In [202]:
sedang

0

In [203]:
tinggi

0.5

In [204]:
hasil = ((tinggi * 100) + (rendah * 30)) + (sedang * 60) / (tinggi + sedang + rendah)
hasil

50.0

In [108]:
'''
kelayakan = 0
pembilang = 0
penyebut = 0
for i in range(1, 101): # 100 titik
  low = min(kelayakan_rendah(i), rendah)
  high = min(kelayakan_tinggi(i), tinggi)
  center = max(low, high)
  pembilang = pembilang + i + center
  penyebut = penyebut + center
 
hasil = pembilang / penyebut
hasil
'''

114.48314606741573